In [2]:

import numpy as np
import cv2
import os
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense


2024-06-15 15:25:23.704043: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-15 15:25:23.932642: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-15 15:25:24.928410: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
dataset_dir = "dataset/dataset/train"
image_paths = []
labels_dict = {}
labels = []
pairs = []
count = 0
for writer_label, writer_folder in enumerate(os.listdir(dataset_dir)):
    count += 1
    if(count > 100):
        break
    writer_folder_path = os.path.join(dataset_dir, writer_folder)
    if not os.path.isdir(writer_folder_path):
        continue

    labels_dict[writer_label] = []

    for img_name in os.listdir(writer_folder_path):
        img_path = os.path.join(writer_folder_path, img_name)
        image_paths.append(img_path)
        labels_dict[writer_label].append(img_path)

In [4]:
def feature_extractor(input_shape=(150, 1000, 1)):
    model = Sequential([
        Conv2D(64, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D((2, 2)),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

In [5]:
f_e = feature_extractor((150, 1000, 1))

/home/ruchitjagodara/.local/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(
I0000 00:00:1718445326.664109   16035 cuda_executor.cc:1032] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-06-15 15:25:26.683170: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2342] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices.

In [6]:
print("Extracting Features! for", len(labels_dict), "writers")
count = 0
X = []
y = []
for label, img_path in labels_dict.items():
    count += 1
    if count > 50: 
        break
    
    for(path) in img_path:
        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        c = img.shape[0]//2
        img = img[c-75:c+75, 600:1600]
        img = cv2.Canny(img, 100, 200)

        img = img / 255.0
        try:
            img = img.reshape(1, 150, 1000, 1)
        except:
            print("Error Reshaping")
            continue
        img = f_e.predict(img)
        img = img.flatten()
        X.append(img)
        y.append(label)
    if count%10==0:print("Features Extracted for", count, "writers")

Extracting Features! for 100 writers
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 232ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 145ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 153ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 143ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 148ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 143ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 148ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 136ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 164ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 166ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s

In [7]:
knn = KNeighborsClassifier(n_neighbors=2)
X = np.array(X)
y = np.array(y)
print(X.shape, y.shape)
knn.fit(X, y)

(401, 535296) (401,)


KNeighborsClassifier(n_neighbors=2)

: 

In [ ]:
y_pred = knn.predict(X)
print(y_pred)
print(y)

In [8]:
knn.score(X, y)